In [121]:
# Imports using Sklearn make shortcut functions
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from datetime import datetime, timedelta

In [122]:
# Print current time
now = datetime.now()
current_time = now.strftime("%Y-%m-%d %H:%M:%S")

In [123]:
# Load csvs from raw_data folder
trends1 = pd.read_csv('../processed_data/LAB_CON_LIB_GRE_BRX_Trends_2004_now.csv', skiprows=1)
trends2 = pd.read_csv('../processed_data/LAB_PLC_SNP_UKI_NAT_Trends_2004_now.csv', skiprows=1)

In [124]:
# Drop duplicate Labour column
trends2.drop(columns='Labour Party: (United Kingdom)',inplace=True)

In [125]:
# Join two trends datasets on 'Month'
trends_merged = pd.merge(trends1,trends2,how='left',on='Month')

In [126]:
# Convert 'Month' field to pd.datetime64[ns] format for joining with polling data
trends_merged['Month'] = pd.to_datetime(trends_merged['Month'])

In [127]:
# Replace <1 values with 0.5 value
trends_merged.replace('<1', 0.5,inplace=True)

In [128]:
# Converting objects (pd.Series) to int dtypes
trends_merged = trends_merged.astype({'Green Party: (United Kingdom)':'int','Reform UK: (United Kingdom)':'int',\
    'Plaid Cymru: (United Kingdom)':'int','Scottish National Party: (United Kingdom)':'int',\
        'UK Independence Party: (United Kingdom)':'int','British National Party: (United Kingdom)':'int'})

In [129]:
# Rename columns to align them more easily with _FC and _ACT columns
trends_merged.rename(columns={'Labour Party: (United Kingdom)': 'LAB_trends',
        'Conservative Party: (United Kingdom)': 'CON_trends',
        'Liberal Democrats: (United Kingdom)': 'LIB_trends',
        'Green Party: (United Kingdom)': 'GRE_trends',
        'Reform UK: (United Kingdom)': 'BRX_trends',
        'Plaid Cymru: (United Kingdom)': 'PLC_trends',
        'Scottish National Party: (United Kingdom)': 'SNP_trends',
        'UK Independence Party: (United Kingdom)': 'UKI_trends',
        'British National Party: (United Kingdom)': 'NAT_trends'}
        ,inplace=True)

In [130]:
# Imports data
data = pd.read_csv('../processed_data/1988_to_2024_combined_clean_polling_and_results.csv')

In [131]:
# Convert date columns to datetime
data['enddate'] = pd.to_datetime(data['enddate'])
data['next_elec_date'] = pd.to_datetime(data['next_elec_date'])
data['startdate'] = pd.to_datetime(data['startdate'])

In [132]:
# Create poll enddate field with YYYY-MM format, so we can join with Trends data
data['enddate_year_month'] = pd.to_datetime(data['enddate']).dt.to_period('M')

In [133]:
# Convert Period datatype to str and then to datetime64
data.enddate_year_month = pd.to_datetime(data.enddate_year_month.astype('str'))

In [134]:
# Merge data with trends data, to get a df with polls, election results and trends
data = pd.merge(data,trends_merged,how='left',left_on='enddate_year_month',right_on='Month')

In [135]:
# Scale trends columns by diving by 100
for column in ['LAB_trends', 'CON_trends', 'LIB_trends',
       'GRE_trends', 'BRX_trends', 'PLC_trends', 'SNP_trends', 'UKI_trends',
       'NAT_trends']:
    data[column] = data[column] / 100

In [136]:
# Handle numerical transformer
num_columns_selector = ['samplesize', 'months_to_elec_weight']
num_transformer = MinMaxScaler()

In [137]:
# Handle categorical transformer
cat_columns_selector = ['rating']
cat_transformer = make_pipeline(OrdinalEncoder(categories = [['F','F+','E-','E','E+','D-','D','D+','C-','C','C+','B-','B','B+','A-']]),MinMaxScaler())

In [138]:
# One-hot encoder for party_in_power
ohe = OneHotEncoder()

In [139]:
# Build the preprocessing pipeline
preproc_pipeline = make_column_transformer(
    (num_transformer, num_columns_selector),
    (cat_transformer, cat_columns_selector),
    (ohe,['party_in_power']),
    remainder='passthrough',
    verbose_feature_names_out=False
)

In [140]:
data['next_elec_date'].value_counts()

next_elec_date
2015-05-07    1930
2024-07-04    1397
2010-05-06     584
1992-04-09     453
2019-12-12     440
1997-05-01     338
2017-06-08     254
2001-06-07     213
2005-05-05      98
Name: count, dtype: int64

In [141]:
# Define election dates
election_date = datetime.strptime('2010-05-06', '%Y-%m-%d')
cutoff_date = election_date - timedelta(days=33)
prediction_date = election_date - timedelta(days=3)

In [142]:
# Split data
data_train = data[data['startdate'] > '2003-12-31']
data_train = data_train[data_train['startdate'] < cutoff_date]
data_test_1 = data[(data['startdate'] >= cutoff_date) & (data['startdate'] < prediction_date)]
data_test = data_test_1[data_test_1['next_elec_date'] == election_date]

In [143]:
# Fit transform preprocessing pipeline to data_train
data_train_processed = preproc_pipeline.fit_transform(data_train)

In [144]:
# Transform preprocessing pipeline to data_test
data_test_processed = preproc_pipeline.transform(data_test)

In [145]:
# Check feature names make sense for data_train
data_train_processed = pd.DataFrame(
    data_train_processed, columns=preproc_pipeline.get_feature_names_out()
)

# Check feature names make sense for data_test
data_test_processed = pd.DataFrame(
    data_test_processed, columns=preproc_pipeline.get_feature_names_out()
)

In [146]:
# Define our X by dropping irrelevant and y columns
X_train = data_train_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])
X_test = data_test_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])

In [147]:
# Build our target matrix
y_train = data_train_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]
y_test = data_test_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]

In [148]:
# Drops y_train actuals where the actual is the actual result we are trying to predict, replaces with NaNs
y_train.loc[y_train['next_elec_date'] == '2019-12-12',
         ['LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']] = np.nan

In [149]:
# Calculates average median value of X_test
averages = X_test.mean()
averages

samplesize                                     0.20788
months_to_elec_weight                         0.853004
rating                                        0.774725
poll_length                                   1.653846
CON_FC                                         0.35141
LAB_FC                                         0.28359
LIB_FC                                        0.259872
BRX_FC                                             NaN
GRE_FC                                             NaN
OTH_FC                                        0.104103
PLC_FC                                             NaN
SNP_FC                                             NaN
UKI_FC                                             NaN
enddate_year_month       2010-04-04 20:18:27.692307712
Month                    2010-04-04 20:18:27.692307712
LAB_trends                                    0.228718
CON_trends                                    0.267692
LIB_trends                                    0.268718
GRE_trends

In [150]:
# Creates imputation values for y_train to impute over actuals we are trying to predict
imputation_values = {
    'CON_ACT': averages['CON_FC'],
    'LAB_ACT': averages['LAB_FC'],
    'LIB_ACT': averages['LIB_FC'],
    'BRX_ACT': averages['BRX_FC'],
    'GRE_ACT': averages['GRE_FC'],
    'OTH_ACT': averages['OTH_FC'],
    'PLC_ACT': averages['PLC_FC'],
    'SNP_ACT': averages['SNP_FC'],
    'UKI_ACT': averages['UKI_FC']
}

In [151]:
# Applies imputation values to y_train
y_train = y_train.fillna(value=imputation_values)


/tmp/ipykernel_51892/3014033841.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = y_train.fillna(value=imputation_values)


In [152]:
# Instantiate the model for each party
models = {}
parties = ['CON', 'LAB', 'LIB', 'BRX', 'GRE', 'SNP', 'UKI', 'PLC', 'OTH']
for party in parties:
    models[party] = XGBRegressor(
        learning_rate=0.3, n_estimators=300, max_depth=3, subsample=0.7,
        objective='reg:squarederror', nthread=-1, enable_categorical=True
    )

In [153]:
X_train.columns

Index(['samplesize', 'months_to_elec_weight', 'rating', 'poll_length',
       'CON_FC', 'LAB_FC', 'LIB_FC', 'BRX_FC', 'GRE_FC', 'OTH_FC', 'PLC_FC',
       'SNP_FC', 'UKI_FC', 'enddate_year_month', 'Month', 'LAB_trends',
       'CON_trends', 'LIB_trends', 'GRE_trends', 'BRX_trends', 'PLC_trends',
       'SNP_trends', 'UKI_trends', 'NAT_trends'],
      dtype='object')

In [154]:
X_train.drop(columns='enddate_year_month', inplace=True)
X_train.drop(columns='Month', inplace=True)


In [155]:
X_test.drop(columns='enddate_year_month', inplace=True)
X_test.drop(columns='Month', inplace=True)

In [156]:
# Train models
X_train_matrix = np.array(X_train)
for party in parties:
    models[party].fit(X_train_matrix, y_train[f'{party}_ACT'])

In [157]:
actuals_2010 = {
    "CON": 0.362,  # Conservative Party
    "LAB": 0.292,  # Labour Party
    "LIB": 0.235,  # Liberal Democrats
    "BRX": 0.3,  # Brexit Party
    "GRE": 0.7,  # Green Party
    "SNP": 0.034,  # Scottish National Party
    "UKI": 0.0000,  # UK Independence Party
    "PLC": 0.006,  # Plaid Cymru
    "OTH": 0.108   # Other parties
}

In [158]:
# Evaluate predictions
X_test_matrix = np.array(X_test)
mean_predictions = {}
for party in parties:
    mean_predictions[party] = models[party].predict(X_test_matrix).mean()


In [159]:
# Print mean predictions
for party in parties:
    print(f"2010 mean prediction for {party}: {mean_predictions[party]}")
    print(f"2010 actual result for {party}  : {actuals_2019[party]}")

2010 mean prediction for CON: 0.3441765010356903
2010 actual result for CON  : 0.4378
2010 mean prediction for LAB: 0.3158215880393982
2010 actual result for LAB  : 0.3267
2010 mean prediction for LIB: 0.2259463220834732
2010 actual result for LIB  : 0.1164
2010 mean prediction for BRX: 0.0
2010 actual result for BRX  : 0.02
2010 mean prediction for GRE: 0.009133817628026009
2010 actual result for GRE  : 0.0254
2010 mean prediction for SNP: 0.01586158759891987
2010 actual result for SNP  : 0.0379
2010 mean prediction for UKI: 0.0267270989716053
2010 actual result for UKI  : 0.0
2010 mean prediction for PLC: 0.005971449427306652
2010 actual result for PLC  : 0.0051
2010 mean prediction for OTH: 0.054219234734773636
2010 actual result for OTH  : 0.0307


In [160]:
# Evaluate last five polls
last_predictions = {}
for party in parties:
    last_predictions[party] = models[party].predict(X_test_matrix)[-5:]

In [161]:
# Print mean of predictions
for party in parties:
    print(f"2019 last prediction for {party}: {last_predictions[party].mean()}")
    print(f"2019 actual result for {party}  : {actuals_2019[party]}")


2019 last prediction for CON: 0.3403889536857605
2019 actual result for CON  : 0.4378
2019 last prediction for LAB: 0.3214220404624939
2019 actual result for LAB  : 0.3267
2019 last prediction for LIB: 0.22485879063606262
2019 actual result for LIB  : 0.1164
2019 last prediction for BRX: 0.0
2019 actual result for BRX  : 0.02
2019 last prediction for GRE: 0.009164822287857533
2019 actual result for GRE  : 0.0254
2019 last prediction for SNP: 0.015776311978697777
2019 actual result for SNP  : 0.0379
2019 last prediction for UKI: 0.02552204392850399
2019 actual result for UKI  : 0.0
2019 last prediction for PLC: 0.006007248070091009
2019 actual result for PLC  : 0.0051
2019 last prediction for OTH: 0.05354524776339531
2019 actual result for OTH  : 0.0307
